<a href="https://colab.research.google.com/github/acse-srm3018/DeeplearningProxy/blob/main/R_U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A few imports before we get started

In [ ]:
import torch
import torch.nn as nn
from core.modules import ResidualConv, Upsample

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

## Create a Residual U-Net as a nn.Module
## Create a feed-forward neural network with the following architecture:

### Encoder
Input layer : Nx;Ny;2
- Convolutional layer:  16 lters of size 3  3  2, stride 2 (Nx=2;Ny=2; 16)
- Convolutional layer: 32 lters of size 3  3  16, stride 1 (Nx=2;Ny=2; 32)
- Convolutional layer: 64 lters of size 3  3  32, stride 2 (Nx=4;Ny=4; 64)
- Convolutional layer: 128 lters of size 3  3  64, stride 1 (Nx=4;Ny=4; 128)
- Residual layer: 128 lters (Nx=4;Ny=4; 128)
- Residual layer: 128 lters (Nx=4;Ny=4; 128)
- Residual layer: 128 lters (Nx=4;Ny=4; 128)

### Decoder
- Residual layer: 128 lters (Nx=4;Ny=4; 128;Nt)
- Residual layer: 128 lters (Nx=4;Ny=4; 128;Nt)
- Residual layer: 128 lters (Nx=4;Ny=4; 128;Nt)
- Transposed convolutional: 128 lters of size 3  3  128, stride 1 (Nx=4;Ny=4; 128;Nt)
- Transposed convolutional: 64 lters of size 3  3  128, stride 2 (Nx=2;Ny=2; 64;Nt)
- Transposed convolutional: 32 lters of size 3  3  64, stride 1 (Nx=2;Ny=2; 32;Nt)
- Tranposed convolutional: 16 lters of size 3  3  32, stride 2 (Nx;Ny; 16;Nt)
- Convolutional layer: 1 lter of size 3  3  16, stride 1 (Nx;Ny; 1;Nt)

- Output Layer Activation: None

In [ ]:
class ResUnet(nn.Module):
    def __init__(self, channel, filters=[64, 128, 256, 512]):
        super(ResUnet, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(channel, filters[0], kernel_size=3, padding=1),
            nn.BatchNorm2d(filters[0]),
            nn.ReLU(),
            nn.Conv2d(filters[0], filters[0], kernel_size=3, padding=1),
        )
        self.input_skip = nn.Sequential(
            nn.Conv2d(channel, filters[0], kernel_size=3, padding=1)
        )

        self.residual_conv_1 = ResidualConv(filters[0], filters[1], 2, 1)
        self.residual_conv_2 = ResidualConv(filters[1], filters[2], 2, 1)

        self.bridge = ResidualConv(filters[2], filters[3], 2, 1)

        self.upsample_1 = Upsample(filters[3], filters[3], 2, 2)
        self.up_residual_conv1 = ResidualConv(filters[3] + filters[2], filters[2], 1, 1)

        self.upsample_2 = Upsample(filters[2], filters[2], 2, 2)
        self.up_residual_conv2 = ResidualConv(filters[2] + filters[1], filters[1], 1, 1)

        self.upsample_3 = Upsample(filters[ 1], filters[1], 2, 2)
        self.up_residual_conv3 = ResidualConv(filters[1] + filters[0], filters[0], 1, 1)

        self.output_layer = nn.Sequential(
            nn.Conv2d(filters[0], 1, 1, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # Encode
        x1 = self.input_layer(x) + self.input_skip(x)
        x2 = self.residual_conv_1(x1)
        x3 = self.residual_conv_2(x2)
        # Bridge
        x4 = self.bridge(x3)
        # Decode
        x4 = self.upsample_1(x4)
        x5 = torch.cat([x4, x3], dim=1)

        x6 = self.up_residual_conv1(x5)

        x6 = self.upsample_2(x6)
        x7 = torch.cat([x6, x2], dim=1)

        x8 = self.up_residual_conv2(x7)

        x8 = self.upsample_3(x8)
        x9 = torch.cat([x8, x1], dim=1)

        x10 = self.up_residual_conv3(x9)

        output = self.output_layer(x10)

        return output